# COUSERA CAPSTONE 
#### My stakeholder want to open a healthy restaurant for young people who eat clean in Toronto City. The problem is where should they put the restaurant. My ideal is they should put the restaurant in a place that near famous universities, gym center, shopping malls and bars. In order to solve that problem, i will take address of universities, gym centers, shopping malls and  bars from Foursquare then use cluster and show them in a map.


In [10]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim #converts an address into latitude and longitude values

import requests 
from pandas.io.json import json_normalize # transform JSON file into a pandas datafame

#Import matplotlib and plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#Import k-means for clustering
from sklearn.cluster import KMeans

import folium #renders map

### *Foursquare Credentials*

In [11]:
CLIENT_ID = 'IFOHJW3UXYMDYLXVE12CQTKNIX30TXPZU31BKKHTNTFPX5WL'
CLIENT_SECRET = '3ICXERSYFLOXBVJKZEKUXGSB5AV0TFJ2V2IUKOLIPUCOF22N'
VERSION = '20180604'
LIMIT = 100

print('Credentials read in!')

Credentials read in!


### *Geospatial Coordinates of Toronto*

In [13]:
geolocator = Nominatim()
address = 'Toronto'

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

<ipython-input-13-de1b6474fa64>:1: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  geolocator = Nominatim()


The geograpical coordinate of the City of Toronto are 43.6534817, -79.3839347.


### *Search for Universities*

In [14]:
#Search for universities
search_query = 'University'
radius = 500

#Create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=IFOHJW3UXYMDYLXVE12CQTKNIX30TXPZU31BKKHTNTFPX5WL&client_secret=3ICXERSYFLOXBVJKZEKUXGSB5AV0TFJ2V2IUKOLIPUCOF22N&ll=43.6534817,-79.3839347&v=20180604&query=University&radius=500&limit=100'

In [15]:
#Send the GET Request 
results = requests.get(url).json()

#Assign relevant part of JSON to venues
venues = results['response']['venues']

#Tranform venues into a dataframe
uni_df = json_normalize(venues)
uni_df.head()

<ipython-input-15-9e41cf044062>:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  uni_df = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress,location.address,location.crossStreet,location.postalCode,location.city,location.state
0,4cd044c29d87224bf129543b,University Centre Gym,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1588234509,False,43.653571,-79.386979,"[{'label': 'display', 'lat': 43.653571, 'lng':...",245,CA,Canada,[Canada],NaN,NaN,NaN,NaN,NaN
1,4d5d275fff226a3110517e69,438 University Ave,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1588234509,False,43.654296,-79.388415,"[{'label': 'display', 'lat': 43.65429570830456...",372,CA,Canada,"[438 University Ave (at Dundas St W), Toronto ...",438 University Ave,at Dundas St W,M5G 2K8,Toronto,ON
2,4c6062b612e5c9286e131f4c,Dundas University Health Clinic,"[{'id': '4bf58dd8d48988d104941735', 'name': 'M...",v-1588234509,False,43.654196,-79.388166,"[{'label': 'display', 'lat': 43.65419587934101...",349,CA,Canada,"[104-438 University Avenue, Toronto ON M4P 2K8...",104-438 University Avenue,NaN,M4P 2K8,Toronto,ON
3,4e6565fb1850e238d7048495,University Avenue Plaza,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",v-1588234509,False,43.652695,-79.387218,"[{'label': 'display', 'lat': 43.65269470686765...",278,CA,Canada,"[University Avenue (btwn Richmond & College), ...",University Avenue,btwn Richmond & College,NaN,Toronto,ON
4,4c45c6c6da2176b09543637a,University Centre,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1588234509,False,43.653907,-79.386764,"[{'label': 'display', 'lat': 43.65390719043468...",232,CA,Canada,"[393 University Ave. (at Armoury St.), Toronto...",393 University Ave.,at Armoury St.,M7A 2K1,Toronto,ON


The Foursquare API returned location data for The University of Toronto and Ryerson University.
No data was returned for York University (ranked 3rd). Therefore, Ontario College of Art and Design University,(ranked 5th), will serve as top 3rd university, as Trent University (ranked 4th), is also not available

### *Clean the University Dataframe*

In [16]:
#Keep columns that include venue name and any associated with location
uni_clean_col = ['name', 'categories'] + [col for col in uni_df.columns if col.startswith('location.')]+ ['id']
uni_clean_df = uni_df.loc[:,uni_clean_col]

#Retrieve venue category 
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#Filter category for each row
uni_clean_df['categories'] = uni_clean_df.apply(get_category_type, axis=1)

#Clean column names by removing the first term
uni_clean_df.columns = [column.split('.')[-1] for column in uni_clean_df.columns]
uni_clean_df.head()

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,formattedAddress,address,crossStreet,postalCode,city,state,id
0,University Centre Gym,Gym / Fitness Center,43.653571,-79.386979,"[{'label': 'display', 'lat': 43.653571, 'lng':...",245,CA,Canada,[Canada],NaN,NaN,NaN,NaN,NaN,4cd044c29d87224bf129543b
1,438 University Ave,Office,43.654296,-79.388415,"[{'label': 'display', 'lat': 43.65429570830456...",372,CA,Canada,"[438 University Ave (at Dundas St W), Toronto ...",438 University Ave,at Dundas St W,M5G 2K8,Toronto,ON,4d5d275fff226a3110517e69
2,Dundas University Health Clinic,Medical Center,43.654196,-79.388166,"[{'label': 'display', 'lat': 43.65419587934101...",349,CA,Canada,"[104-438 University Avenue, Toronto ON M4P 2K8...",104-438 University Avenue,NaN,M4P 2K8,Toronto,ON,4c6062b612e5c9286e131f4c
3,University Avenue Plaza,Plaza,43.652695,-79.387218,"[{'label': 'display', 'lat': 43.65269470686765...",278,CA,Canada,"[University Avenue (btwn Richmond & College), ...",University Avenue,btwn Richmond & College,NaN,Toronto,ON,4e6565fb1850e238d7048495
4,University Centre,Office,43.653907,-79.386764,"[{'label': 'display', 'lat': 43.65390719043468...",232,CA,Canada,"[393 University Ave. (at Armoury St.), Toronto...",393 University Ave.,at Armoury St.,M7A 2K1,Toronto,ON,4c45c6c6da2176b09543637a


In [17]:
#Delete unnecessary columns
uni_clean_df2= uni_clean_df.drop(['cc', 'city', 'country', 'crossStreet', 'distance','formattedAddress', 'id','labeledLatLngs'], axis=1)
uni_clean_df2.head()

,name,categories,lat,lng,address,postalCode,state
0,University Centre Gym,Gym / Fitness Center,43.653571,-79.386979,NaN,NaN,NaN
1,438 University Ave,Office,43.654296,-79.388415,438 University Ave,M5G 2K8,ON
2,Dundas University Health Clinic,Medical Center,43.654196,-79.388166,104-438 University Avenue,M4P 2K8,ON
3,University Avenue Plaza,Plaza,43.652695,-79.387218,University Avenue,NaN,ON
4,University Centre,Office,43.653907,-79.386764,393 University Ave.,M7A 2K1,ON


In [18]:
#Delete rows with NaN values
uni_clean_df3 = uni_clean_df2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False) 
uni_clean_df3

,name,categories,lat,lng,address,postalCode,state
1,438 University Ave,Office,43.654296,-79.388415,438 University Ave,M5G 2K8,ON
2,Dundas University Health Clinic,Medical Center,43.654196,-79.388166,104-438 University Avenue,M4P 2K8,ON
4,University Centre,Office,43.653907,-79.386764,393 University Ave.,M7A 2K1,ON
6,393 University Ave,Office,43.653752,-79.388150,393 University Ave,M5G 1E6,ON
9,480 University Ave,Office,43.655099,-79.388648,480 University Ave,M5G 1V2,ON
13,Ontario College of Art and Design University (...,University,43.652803,-79.391074,100 McCaul St,M5T 1W1,ON
14,425 University Avenue,Office,43.653769,-79.383949,425 University Avenue,M5G 1T6,ON
27,University Health Network Spiritual Care Depar...,Spiritual Center,43.658209,-79.384962,700 Bay St,M5G 0A2,ON
30,University Settlement Recreation Centre,College Gym,43.652139,-79.391477,23 Grange Rd.,M5T 1C3,ON
31,"Regus - Ontario, Toronto - University & Dundas",Business Service,43.654400,-79.387900,"439 University Avenue, 5th Floor",M5G 1Y8,ON


The location data does not display direct location information for The University of Toronto. However, the addresses that include ('University Ave') are within The University of Toronto campus boundaries.
For purposes of further analysis, Dundas & University in the 'Intersection' category will be used as the University of Toronto's main address. This replacement location identifier is approx. 3 minutes driving distance from the offical campus address.


### *Clean and Isolate Top Universities Dataframe*

In [42]:
#Choose top 3 university locations by selecting where category is equal to Intersection or University
array= ['Intersection', 'University']
topuni_df= uni_clean_df3.loc[uni_clean_df3['categories'].isin(array)]
topuni_df
#Replace Ontario College of Art and Design University with 'OCAD' University
topuni_df.loc[13, 'name'] = 'OCAD University'

#Replace 'Dundas & University' name with 'University of Toronto'
topuni_df.loc[43,'name'] = 'University of Toronto'

#Replace category 'Intersection' with 'University'
topuni_df.loc[43,'categories'] = 'University'

topuni_df


c:\users\hp\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,name,categories,lat,lng,address,postalCode,state
13,OCAD University,University,43.652803,-79.391074,100 McCaul St,M5T 1W1,ON
43,University of Toronto,University,43.654620,-79.389150,University Ave,M5G 1V1,ON
44,Ryerson University,University,43.657935,-79.380490,350 Victoria St.,M5B 2K3,ON


### *Search for Gyms*

In [20]:
#Search for Gyms
search_query = ['Fitness','Training']
radius = 500

#Create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

"https://api.foursquare.com/v2/venues/search?client_id=IFOHJW3UXYMDYLXVE12CQTKNIX30TXPZU31BKKHTNTFPX5WL&client_secret=3ICXERSYFLOXBVJKZEKUXGSB5AV0TFJ2V2IUKOLIPUCOF22N&ll=43.6534817,-79.3839347&v=20180604&query=['Fitness', 'Training']&radius=500&limit=100"

In [21]:
#Send the GET Request 
gym_results = requests.get(url).json()

#Assign relevant part of JSON to venues
venues = gym_results['response']['venues']

#Tranform venues into a dataframe
gym_df = json_normalize(venues)
gym_df.head()

<ipython-input-21-43f5fa3f596e>:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  gym_df = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,venuePage.id
0,4ff8e46fe4b03705cefc7075,Fitness Centre,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1588234510,False,525 Bay St.,in Marriott Downtown,43.654690,-79.381739,"[{'label': 'display', 'lat': 43.65469045595554...",222,M5G 2L2,CA,Toronto,ON,Canada,"[525 Bay St. (in Marriott Downtown), Toronto O...",NaN,NaN
1,50885719498ea7b5aab3a74c,GoodLife Fitness Toronto Bell Trinity Centre,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1588234510,False,483 Bay St,NaN,43.653436,-79.382314,"[{'label': 'display', 'lat': 43.653436, 'lng':...",130,M5G 2C9,CA,Toronto,ON,Canada,"[483 Bay St, Toronto ON M5G 2C9, Canada]",NaN,NaN
2,4b5243a7f964a5209d7327e3,Fitness Centre,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1588234510,False,Sheraton Centre,123 Queen St W,43.650985,-79.384002,"[{'label': 'display', 'lat': 43.65098505166592...",277,M5H 2M9,CA,Toronto,ON,Canada,"[Sheraton Centre (123 Queen St W), Toronto ON ...",NaN,NaN
3,53451cff498e555a4e340937,doubletree fitness center (Doubletree Fitness ...,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1588234510,False,108 Chestnut Street,Dundas St W,43.654603,-79.386204,"[{'label': 'display', 'lat': 43.65460313972433...",221,M5G 1R3,CA,Toronto,ON,Canada,"[108 Chestnut Street (Dundas St W), Toronto ON...",NaN,NaN
4,5ba17f68911fc4002c36a07d,Busy Bee Fitness Experts,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1588234510,False,319 Merton Street,Mount Pleasant,43.653944,-79.383370,"[{'label': 'display', 'lat': 43.65394410274548...",68,M4S 1A5,CA,Toronto,ON,Canada,"[319 Merton Street (Mount Pleasant), Toronto O...",NaN,NaN


### *Clean the Gym Dataframe*

In [22]:
#Keep columns that include venue name and any associated with location
gym_clean_col = ['name', 'categories'] + [col for col in gym_df.columns if col.startswith('location.')]+ ['id']
gym_clean_df = gym_df.loc[:,gym_clean_col]

#Retrieve venue category 
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#Filter category for each row
gym_clean_df['categories'] = gym_clean_df.apply(get_category_type, axis=1)

#Clean column names by removing the first term
gym_clean_df.columns = [column.split('.')[-1] for column in gym_clean_df.columns]
gym_clean_df.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Fitness Centre,Gym,525 Bay St.,in Marriott Downtown,43.654690,-79.381739,"[{'label': 'display', 'lat': 43.65469045595554...",222,M5G 2L2,CA,Toronto,ON,Canada,"[525 Bay St. (in Marriott Downtown), Toronto O...",NaN,4ff8e46fe4b03705cefc7075
1,GoodLife Fitness Toronto Bell Trinity Centre,Gym,483 Bay St,NaN,43.653436,-79.382314,"[{'label': 'display', 'lat': 43.653436, 'lng':...",130,M5G 2C9,CA,Toronto,ON,Canada,"[483 Bay St, Toronto ON M5G 2C9, Canada]",NaN,50885719498ea7b5aab3a74c
2,Fitness Centre,Gym,Sheraton Centre,123 Queen St W,43.650985,-79.384002,"[{'label': 'display', 'lat': 43.65098505166592...",277,M5H 2M9,CA,Toronto,ON,Canada,"[Sheraton Centre (123 Queen St W), Toronto ON ...",NaN,4b5243a7f964a5209d7327e3
3,doubletree fitness center (Doubletree Fitness ...,Gym / Fitness Center,108 Chestnut Street,Dundas St W,43.654603,-79.386204,"[{'label': 'display', 'lat': 43.65460313972433...",221,M5G 1R3,CA,Toronto,ON,Canada,"[108 Chestnut Street (Dundas St W), Toronto ON...",NaN,53451cff498e555a4e340937
4,Busy Bee Fitness Experts,Gym / Fitness Center,319 Merton Street,Mount Pleasant,43.653944,-79.383370,"[{'label': 'display', 'lat': 43.65394410274548...",68,M4S 1A5,CA,Toronto,ON,Canada,"[319 Merton Street (Mount Pleasant), Toronto O...",NaN,5ba17f68911fc4002c36a07d


In [23]:
#Delete unnecessary columns
gym_clean_df2= gym_clean_df.drop(['cc', 'city', 'country', 'crossStreet', 'distance','formattedAddress', 'id','labeledLatLngs', 'neighborhood'], axis=1)
gym_clean_df2.head()

,name,categories,address,lat,lng,postalCode,state
0,Fitness Centre,Gym,525 Bay St.,43.654690,-79.381739,M5G 2L2,ON
1,GoodLife Fitness Toronto Bell Trinity Centre,Gym,483 Bay St,43.653436,-79.382314,M5G 2C9,ON
2,Fitness Centre,Gym,Sheraton Centre,43.650985,-79.384002,M5H 2M9,ON
3,doubletree fitness center (Doubletree Fitness ...,Gym / Fitness Center,108 Chestnut Street,43.654603,-79.386204,M5G 1R3,ON
4,Busy Bee Fitness Experts,Gym / Fitness Center,319 Merton Street,43.653944,-79.383370,M4S 1A5,ON


In [24]:
#Delete rows with NaN values
gym_clean_df3 = gym_clean_df2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False) 
gym_clean_df3
gym_clean_df4 = gym_clean_df3.replace({'categories': r'^Gym / Fitness Center'}, {'categories':'Gym'},regex=True)
gym_clean_df3 = gym_clean_df4.replace({'categories': r'Athletics & Sports'}, {'categories':'Gym'},regex=True)

gym_clean_df3

,name,categories,address,lat,lng,postalCode,state
0,Fitness Centre,Gym,525 Bay St.,43.654690,-79.381739,M5G 2L2,ON
1,GoodLife Fitness Toronto Bell Trinity Centre,Gym,483 Bay St,43.653436,-79.382314,M5G 2C9,ON
2,Fitness Centre,Gym,Sheraton Centre,43.650985,-79.384002,M5H 2M9,ON
3,doubletree fitness center (Doubletree Fitness ...,Gym,108 Chestnut Street,43.654603,-79.386204,M5G 1R3,ON
4,Busy Bee Fitness Experts,Gym,319 Merton Street,43.653944,-79.383370,M4S 1A5,ON
5,Google Toronto Fitness Centre,Gym,"111 Richmond St W, 11F",43.650319,-79.383782,M5H 2G4,ON
8,Flex - Personal Training Delivered to You,Gym,250 Yonge St,43.654196,-79.381714,M5B 2M6,ON
10,GoodLife Fitness Toronto 137 Yonge Street,Gym,137 Yonge St,43.651242,-79.378068,M5C 1W6,ON
11,Great West Life Fitness Centre,Gym,02 - 190 Simcoe St,43.652721,-79.381328,M5T 3M3,ON
12,Medx Precision Fitness,Gym,100-80 Richmond St W,43.651429,-79.382356,M5H 2A4,ON


In [25]:
#Set all categories's values to Gym
array= ['Gym', 'Gym / Fitness Center', 'Athletics & Sports']
gym_df= gym_clean_df3.loc[gym_clean_df3['categories'].isin(array)]
gym_df

,name,categories,address,lat,lng,postalCode,state
0,Fitness Centre,Gym,525 Bay St.,43.654690,-79.381739,M5G 2L2,ON
1,GoodLife Fitness Toronto Bell Trinity Centre,Gym,483 Bay St,43.653436,-79.382314,M5G 2C9,ON
2,Fitness Centre,Gym,Sheraton Centre,43.650985,-79.384002,M5H 2M9,ON
3,doubletree fitness center (Doubletree Fitness ...,Gym,108 Chestnut Street,43.654603,-79.386204,M5G 1R3,ON
4,Busy Bee Fitness Experts,Gym,319 Merton Street,43.653944,-79.383370,M4S 1A5,ON
5,Google Toronto Fitness Centre,Gym,"111 Richmond St W, 11F",43.650319,-79.383782,M5H 2G4,ON
8,Flex - Personal Training Delivered to You,Gym,250 Yonge St,43.654196,-79.381714,M5B 2M6,ON
10,GoodLife Fitness Toronto 137 Yonge Street,Gym,137 Yonge St,43.651242,-79.378068,M5C 1W6,ON
11,Great West Life Fitness Centre,Gym,02 - 190 Simcoe St,43.652721,-79.381328,M5T 3M3,ON
12,Medx Precision Fitness,Gym,100-80 Richmond St W,43.651429,-79.382356,M5H 2A4,ON


### *Search for Bars*

In [26]:
#Search for Bars
search_query = 'Bar'
radius = 500

#Create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=IFOHJW3UXYMDYLXVE12CQTKNIX30TXPZU31BKKHTNTFPX5WL&client_secret=3ICXERSYFLOXBVJKZEKUXGSB5AV0TFJ2V2IUKOLIPUCOF22N&ll=43.6534817,-79.3839347&v=20180604&query=Bar&radius=500&limit=100'

In [27]:
#Send the GET Request 
bar_results = requests.get(url).json()

#Assign relevant part of JSON to venues
venues = bar_results['response']['venues']

#Tranform venues into a dataframe
bar_df = json_normalize(venues)
bar_df.head()

<ipython-input-27-04ae01287a6b>:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  bar_df = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,venuePage.id
0,4af8da92f964a5204a1022e3,Blue Stone Grill & Bar,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",v-1588234511,False,372 Bay St.,at Richmond St.,43.651187,-79.381139,"[{'label': 'display', 'lat': 43.65118713330148...",340,M5H 1M7,CA,Toronto,ON,Canada,"[372 Bay St. (at Richmond St.), Toronto ON M5H...",NaN,NaN
1,4ad949def964a520991921e3,Bar+ Karaoke Lounge,"[{'id': '4bf58dd8d48988d120941735', 'name': 'K...",v-1588234511,False,360 Yonge St.,at Elm St.,43.658338,-79.381902,"[{'label': 'display', 'lat': 43.65833815125888...",564,M5B 1S5,CA,Toronto,ON,Canada,"[360 Yonge St. (at Elm St.), Toronto ON M5B 1S...",NaN,NaN
2,57dd99cb498ee67580d16390,Bar Verde,"[{'id': '4bf58dd8d48988d157941735', 'name': 'N...",v-1588234511,False,260 Yonge Street,NaN,43.654837,-79.380742,"[{'label': 'display', 'lat': 43.654837, 'lng':...",298,M5B 2L9,CA,Toronto,ON,Canada,"[260 Yonge Street, Toronto ON M5B 2L9, Canada]",NaN,NaN
3,4b3db5e4f964a520709625e3,Quinn's Steakhouse & Bar,"[{'id': '4bf58dd8d48988d1cc941735', 'name': 'S...",v-1588234511,False,96 Richmond Street West,at Bay,43.651197,-79.382976,"[{'label': 'display', 'lat': 43.65119745750837...",265,M5H 2A3,CA,Toronto,ON,Canada,"[96 Richmond Street West (at Bay), Toronto ON ...",Financial District,90490783
4,50ca02be245f2d4aa8c2ab5b,Barristers Bar,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1588234511,False,145 Richmond Street West,Hilton Toronto,43.649797,-79.385807,"[{'label': 'display', 'lat': 43.64979669604817...",437,M5H 2L2,CA,Toronto,ON,Canada,"[145 Richmond Street West (Hilton Toronto), To...",NaN,NaN


### *Clean the Bar Dataframe*

In [28]:
#Keep columns that include venue name and any associated with location
bar_clean_col = ['name', 'categories'] + [col for col in bar_df.columns if col.startswith('location.')]+ ['id']
bar_clean_df = bar_df.loc[:,bar_clean_col]

#Retrieve venue category 
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#Filter category for each row
bar_clean_df['categories'] = bar_clean_df.apply(get_category_type, axis=1)

#Clean column names by removing the first term
bar_clean_df.columns = [column.split('.')[-1] for column in bar_clean_df.columns]
bar_clean_df.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Blue Stone Grill & Bar,Pub,372 Bay St.,at Richmond St.,43.651187,-79.381139,"[{'label': 'display', 'lat': 43.65118713330148...",340,M5H 1M7,CA,Toronto,ON,Canada,"[372 Bay St. (at Richmond St.), Toronto ON M5H...",NaN,4af8da92f964a5204a1022e3
1,Bar+ Karaoke Lounge,Karaoke Bar,360 Yonge St.,at Elm St.,43.658338,-79.381902,"[{'label': 'display', 'lat': 43.65833815125888...",564,M5B 1S5,CA,Toronto,ON,Canada,"[360 Yonge St. (at Elm St.), Toronto ON M5B 1S...",NaN,4ad949def964a520991921e3
2,Bar Verde,New American Restaurant,260 Yonge Street,NaN,43.654837,-79.380742,"[{'label': 'display', 'lat': 43.654837, 'lng':...",298,M5B 2L9,CA,Toronto,ON,Canada,"[260 Yonge Street, Toronto ON M5B 2L9, Canada]",NaN,57dd99cb498ee67580d16390
3,Quinn's Steakhouse & Bar,Steakhouse,96 Richmond Street West,at Bay,43.651197,-79.382976,"[{'label': 'display', 'lat': 43.65119745750837...",265,M5H 2A3,CA,Toronto,ON,Canada,"[96 Richmond Street West (at Bay), Toronto ON ...",Financial District,4b3db5e4f964a520709625e3
4,Barristers Bar,Restaurant,145 Richmond Street West,Hilton Toronto,43.649797,-79.385807,"[{'label': 'display', 'lat': 43.64979669604817...",437,M5H 2L2,CA,Toronto,ON,Canada,"[145 Richmond Street West (Hilton Toronto), To...",NaN,50ca02be245f2d4aa8c2ab5b


In [29]:
#Delete unnecessary columns
bar_clean_df2= bar_clean_df.drop(['cc', 'city', 'country', 'crossStreet', 'distance','formattedAddress', 'id','labeledLatLngs', 'neighborhood'], axis=1)
bar_clean_df2.head()

,name,categories,address,lat,lng,postalCode,state
0,Blue Stone Grill & Bar,Pub,372 Bay St.,43.651187,-79.381139,M5H 1M7,ON
1,Bar+ Karaoke Lounge,Karaoke Bar,360 Yonge St.,43.658338,-79.381902,M5B 1S5,ON
2,Bar Verde,New American Restaurant,260 Yonge Street,43.654837,-79.380742,M5B 2L9,ON
3,Quinn's Steakhouse & Bar,Steakhouse,96 Richmond Street West,43.651197,-79.382976,M5H 2A3,ON
4,Barristers Bar,Restaurant,145 Richmond Street West,43.649797,-79.385807,M5H 2L2,ON


In [30]:
#Delete rows with NaN values
bar_clean_df3 = bar_clean_df2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False) 
bar_clean_df3

,name,categories,address,lat,lng,postalCode,state
0,Blue Stone Grill & Bar,Pub,372 Bay St.,43.651187,-79.381139,M5H 1M7,ON
1,Bar+ Karaoke Lounge,Karaoke Bar,360 Yonge St.,43.658338,-79.381902,M5B 1S5,ON
2,Bar Verde,New American Restaurant,260 Yonge Street,43.654837,-79.380742,M5B 2L9,ON
3,Quinn's Steakhouse & Bar,Steakhouse,96 Richmond Street West,43.651197,-79.382976,M5H 2A3,ON
4,Barristers Bar,Restaurant,145 Richmond Street West,43.649797,-79.385807,M5H 2L2,ON
5,Pantages Lounge & Bar,Cocktail Bar,200 Victoria St.,43.654493,-79.379000,M5B 1W8,ON
7,St. Louis Bar & Grill,Bar,595 Bay St #A09,43.656562,-79.382737,M5G 2C2,ON
8,Milestones Grill + Bar,American Restaurant,400 - 10 Dundas St E,43.656345,-79.380840,M5B 2G9,ON
9,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
10,The Keg Steakhouse + Bar - York Street,Restaurant,165 York St,43.649987,-79.384103,M5H 3R8,ON


In [37]:
#Set all categories's values to Bar
array= ['Bar', 'Sports Bar', 'Nightclub', 'Cocktail Bar', 'Hotel Bar', 'Pub', 'Jazz Club', 'Karaoke Bar']
bar_df= bar_clean_df3.loc[bar_clean_df3['categories'].isin(array)]
bar_df.iloc[:,1] = 'Bar'
bar_df

c:\users\hp\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,name,categories,address,lat,lng,postalCode,state
0,Blue Stone Grill & Bar,Bar,372 Bay St.,43.651187,-79.381139,M5H 1M7,ON
1,Bar+ Karaoke Lounge,Bar,360 Yonge St.,43.658338,-79.381902,M5B 1S5,ON
5,Pantages Lounge & Bar,Bar,200 Victoria St.,43.654493,-79.379000,M5B 1W8,ON
7,St. Louis Bar & Grill,Bar,595 Bay St #A09,43.656562,-79.382737,M5G 2C2,ON
9,The Rex Hotel Jazz & Blues Bar,Bar,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
19,St. Louis Bar & Grill,Bar,"595 Bay St.,Unit A08",43.655992,-79.383463,M5G 2C2,ON
20,Bar Senator,Bar,249 Victoria Street,43.655380,-79.379088,M5B 1V8,ON
25,Bar+ Karaoke,Bar,360 Yonge St,43.657968,-79.381907,M6S 1Z9,ON
29,Bar Adelaide,Bar,325 Bay Street,43.649800,-79.380160,M5H,ON
30,T|Bar,Bar,33 Gerrard Street West,43.658727,-79.383047,M5G 1Z4,ON


### *Shopping malls*

In [43]:
#Search for shopping malls
search_query = 'Shopping'
radius = 1000

#Create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=IFOHJW3UXYMDYLXVE12CQTKNIX30TXPZU31BKKHTNTFPX5WL&client_secret=3ICXERSYFLOXBVJKZEKUXGSB5AV0TFJ2V2IUKOLIPUCOF22N&ll=43.6534817,-79.3839347&v=20180604&query=Shopping&radius=1000&limit=100'

In [44]:
#Send the GET Request 
mall_results = requests.get(url).json()

#Assign relevant part of JSON to venues
venues = mall_results['response']['venues']

#Tranform venues into a dataframe
mall_df = json_normalize(venues)
mall_df

<ipython-input-44-f1d4043f789f>:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  mall_df = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood
0,5840cafe06f1a34af80cc609,Saks Fifth Avenue Club - Personal Shopping,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",v-1588235594,False,176 Yonge Street,Queen Street West,43.651810,-79.379192,"[{'label': 'display', 'lat': 43.65181, 'lng': ...",424,M5C 2L7,CA,Toronto,ON,Canada,"[176 Yonge Street (Queen Street West), Toronto...",NaN
1,4b9e86dff964a520e6eb36e3,TD Centre Shopping Concourse,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1588235594,False,66 Wellington St W,btw York & Bay St,43.647184,-79.380932,"[{'label': 'display', 'lat': 43.64718424611984...",741,M5K 1A1,CA,Toronto,ON,Canada,"[66 Wellington St W (btw York & Bay St), Toron...",NaN
2,4ad77a12f964a520260b21e3,CF Toronto Eaton Centre,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1588235594,False,220 Yonge St,btwn Queen & Dundas,43.654540,-79.380677,NaN,287,M5B 2H1,CA,Toronto,ON,Canada,"[220 Yonge St (btwn Queen & Dundas), Toronto O...","Downtown Toronto, Toronto, ON"


Clean the Shopping Mall Dataframe

In [45]:
#Keep columns that include venue name and any associated with location
mall_clean_col = ['name', 'categories'] + [col for col in mall_df.columns if col.startswith('location.')]+ ['id']
mall_clean_df = mall_df.loc[:,mall_clean_col]

#Retrieve venue category 
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#Filter category for each row
mall_clean_df['categories'] = mall_clean_df.apply(get_category_type, axis=1)

#Clean column names by removing the first term
mall_clean_df.columns = [column.split('.')[-1] for column in mall_clean_df.columns]
mall_clean_df.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,Queen Street West,43.651810,-79.379192,"[{'label': 'display', 'lat': 43.65181, 'lng': ...",424,M5C 2L7,CA,Toronto,ON,Canada,"[176 Yonge Street (Queen Street West), Toronto...",NaN,5840cafe06f1a34af80cc609
1,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,btw York & Bay St,43.647184,-79.380932,"[{'label': 'display', 'lat': 43.64718424611984...",741,M5K 1A1,CA,Toronto,ON,Canada,"[66 Wellington St W (btw York & Bay St), Toron...",NaN,4b9e86dff964a520e6eb36e3
2,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,btwn Queen & Dundas,43.654540,-79.380677,NaN,287,M5B 2H1,CA,Toronto,ON,Canada,"[220 Yonge St (btwn Queen & Dundas), Toronto O...","Downtown Toronto, Toronto, ON",4ad77a12f964a520260b21e3


In [46]:
#Delete unnecessary columns
mall_clean_df2= mall_clean_df.drop(['cc', 'city', 'country', 'crossStreet', 'distance','formattedAddress', 'id','labeledLatLngs', 'neighborhood'], axis=1)
mall_clean_df2.head()


,name,categories,address,lat,lng,postalCode,state
0,Saks Fifth Avenue Club - Personal Shopping,Department Store,176 Yonge Street,43.651810,-79.379192,M5C 2L7,ON
1,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON
2,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654540,-79.380677,M5B 2H1,ON


In [48]:

#Choose shopping mall locations by selecting where category is equal to Shopping Mall
array= ['Shopping Mall']
mall_df= mall_clean_df2.loc[mall_clean_df2['categories'].isin(array)]
mall_df

,name,categories,address,lat,lng,postalCode,state
1,TD Centre Shopping Concourse,Shopping Mall,66 Wellington St W,43.647184,-79.380932,M5K 1A1,ON
2,CF Toronto Eaton Centre,Shopping Mall,220 Yonge St,43.654540,-79.380677,M5B 2H1,ON


### *Visualization*   
Merge the University, Gym, Bar and Shopping Mall dataframes in preparation for cluster map

In [49]:
#Create new dataframe of universities, gyms, bars and shopping malls
frames = [topuni_df, bar_df, mall_df, gym_df]
restaurant_df = pd.concat(frames, sort=True)
restaurant_df

,address,categories,lat,lng,name,postalCode,state
13,100 McCaul St,University,43.652803,-79.391074,OCAD University,M5T 1W1,ON
43,University Ave,University,43.654620,-79.389150,University of Toronto,M5G 1V1,ON
44,350 Victoria St.,University,43.657935,-79.380490,Ryerson University,M5B 2K3,ON
0,372 Bay St.,Bar,43.651187,-79.381139,Blue Stone Grill & Bar,M5H 1M7,ON
1,360 Yonge St.,Bar,43.658338,-79.381902,Bar+ Karaoke Lounge,M5B 1S5,ON
5,200 Victoria St.,Bar,43.654493,-79.379000,Pantages Lounge & Bar,M5B 1W8,ON
7,595 Bay St #A09,Bar,43.656562,-79.382737,St. Louis Bar & Grill,M5G 2C2,ON
9,194 Queen St W,Bar,43.650505,-79.388577,The Rex Hotel Jazz & Blues Bar,M5V 1Z1,ON
19,"595 Bay St.,Unit A08",Bar,43.655992,-79.383463,St. Louis Bar & Grill,M5G 2C2,ON
20,249 Victoria Street,Bar,43.655380,-79.379088,Bar Senator,M5B 1V8,ON


In [50]:
#Create map of clusters using latitude and longitude
latitude= 43.6534817
longitude= -79.3839347

kclusters=7

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

#Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
rainbow = ["Blue","Cyan","Fuchsia","Purple"]

#Add markers to the map and assign marker colors
markers_colors = []
for lat, lon, poi, cluster in zip(restaurant_df['lat'], restaurant_df['lng'],restaurant_df['categories'], restaurant_df['address']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[np.select(
                                [poi=="Gym", 
                                 poi=="University", 
                                 poi=="Bar", 
                                 poi=="Shopping Mall"], 
                                [0,1,2,3],default=0
                               )
                     ],
        fill=True,
        fill_color=rainbow[np.select(
                                [poi=="Gym", 
                                 poi=="University",
                                 poi=="Bar", 
                                 poi=="Shopping Mall"], 
                                [0,1,2,3],default=0
                               )
                     ],
        fill_opacity=0.7).add_to(map_clusters)  
    
       
map_clusters

#### Insights
#### Given the results of the cluster map, the restaurant may see more traffic if they open near Ryerson University. In comparison to the University of Toronto and OCAD University, Ryerson has more near by venues. It should also be noted that out of the three universities, Ryerson's location is the closet to the nearest shopping mall